# Fine-tune Llama 2 with DPO

In [4]:
from datasets import load_dataset

def return_prompt_and_responses(samples):# -> Dict[str, str, str]:
    return {
        "prompt": [
            "Question: " + question + "\n\nAnswer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }
dataset = load_dataset(
    "lvwerra/stack-exchange-paired",
    split="train",
    data_dir="data/rl"
)
original_columns = dataset.column_names
dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)
dataset

/home/matthias/Desktop/Compute-Optimal_LoRA_Adapters_for_Causal-LM/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|█████████████████████| 20/20 [00:00<00:00, 51.82it/s]
Generating train split: 7435908 examples [00:23, 315901.42 examples/s]
Map: 100%|█████████████████| 7435908/7435908 [01:06<00:00, 111641.38 examples/s]


Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 7435908
})

In [ ]:
from transformers import DPOTrainer
dpo_trainer = DPOTrainer(
    model,                 # base model from SFT pipeline
    model_ref,             # typically a copy of the SFT trained base model
    beta=0.1,              # temperature hyperparameter of DPO
    train_dataset=dataset, # dataset prepared above
    tokenizer=tokenizer,   # tokenizer
    args=training_args,    # training arguments e.g. batch size, lr, etc.
)
dpo_trainer